In [47]:
DATABASE_PATH = '../database/abundances_new.db'
element_list = ['Fe', 'Ti', 'Ca', 'Si', 'Al', 'Mg', 'Na']

In [48]:
import json
def query_db(query, params=(), limit=None):
    db_path = DATABASE_PATH
    connection = sqlite3.connect(db_path)

    # Add LIMIT clause if a limit is specified
    if limit is not None:
        query += f" LIMIT {limit}"
        
    # Using pandas to read the query result directly into a DataFrame
    df = pd.read_sql_query(query, connection, params=params)

    connection.close()

    # Convert DataFrame to a list of dictionaries
    result_list = df.to_dict(orient='records')

    return result_list


## fetching single element

In [49]:

# Example of querying the database for a specific element
element = "Si"  # Example element (replace with your query parameter)
# date = None # Example date (can be None or a valid date)
date = None  # Example date (can be None or a valid date)
# date = "2021-04-22"  # Example date (can be None or a valid date)
# date = "2020-01-02"  # Example date (can be None or a valid date)

FETCH_LIMIT  = None
# Build the query based on the presence of 'date'
if date:
    query = "SELECT * FROM abundances WHERE element = ? AND date = ?"
    data = (element, date)
else:
    query = "SELECT * FROM abundances WHERE element = ?"
    data = (element,)


# Start the timer to measure the query execution time
start_time = time.time()

result = query_db(query, data,FETCH_LIMIT )


end_time = time.time()
# Print the time taken to perform the query
print(f"time to fetch : {end_time - start_time:.4f} seconds")
# Get the result from the query_db function

# Print the result in JSON-like format (list of dictionaries)
# print(json.dumps(result, indent=4))

time to fetch : 2.1485 seconds


In [50]:
len(result)

250000

In [51]:
import numpy

In [52]:
import time
from sklearn.neighbors import KDTree
import numpy as np

points = result

# # List of points, where each point is a dictionary
# points = [
#     {'element': 'Si', 'date': '2020-01-01', 'abundance': 17.0, 'lat': 66.988234, 'long': 22.479195, 'year': 2024, 'day': 5, 'hour': 14, 'min': 9, 'sec': 57},
#     {'element': 'O', 'date': '2021-05-10', 'abundance': 20.0, 'lat': 66.988500, 'long': 22.479800, 'year': 2025, 'day': 10, 'hour': 15, 'min': 10, 'sec': 30},
#     {'element': 'Mg', 'date': '2022-08-12', 'abundance': 19.5, 'lat': 66.987000, 'long': 22.478000, 'year': 2023, 'day': 12, 'hour': 13, 'min': 8, 'sec': 10},
#     # Add more points...
# ]

# Start the timer to measure the query execution time
start_time = time.time()

# Extract lat, long as 2D NumPy array (lat, long)
points_array = np.array([(point['lat'], point['long']) for point in points])

end_time = time.time()
# Print the time taken to perform the query
print(f"Array creation time {end_time - start_time:.4f} seconds")
start_time = time.time()
# Create a KDTree with the points
tree = KDTree(points_array)
print(f"Tree creation time {end_time - start_time:.4f} seconds")

# End the timer after the query is completed
end_time = time.time()

# Function to fetch nearby points within a radius
def get_nearby_points(query_point, radius_degrees):
    # Convert query_point to a NumPy array
    query_array = np.array([query_point])

    # Query the tree to find points within the radius (in degrees)
    indices = tree.query_radius(query_array, r=radius_degrees)

    # Fetch the actual nearby points using the indices
    nearby_points = [points[i] for i in indices[0]]

    return nearby_points

# Example query for nearby points
query_point = (66.988234, 22.479195)  # A point to query (lat, long)
radius = 2.0  # Radius in degrees (adjust as needed)

# Start the timer to measure the query execution time
start_time = time.time()

# Get points within the radius
nearby_points = get_nearby_points(query_point, radius)

# End the timer after the query is completed
end_time = time.time()

print(f"{len(nearby_points)} found")
# # Print the nearby points
# for point in nearby_points:
#     print(point)

# Print the time taken to perform the query
print(f"Query executed in {end_time - start_time:.4f} seconds")


Array creation time 0.0834 seconds
Tree creation time 0.0000 seconds
53 found
Query executed in 0.0250 seconds


## fetching all points

In [80]:
del result

In [81]:
query = "SELECT * FROM abundances"
data = []

In [82]:
# Start the timer to measure the query execution time
start_time = time.time()

result = query_db(query, data,FETCH_LIMIT )


end_time = time.time()
# Print the time taken to perform the query
print(f"time to fetch : {end_time - start_time:.4f} seconds")

time to fetch : 11.2312 seconds


In [83]:
len(result) / 250000

7.0

In [84]:
import sys


# Get the size of the variable
size_in_bytes = sys.getsizeof(result)

print(f"Size of my_list: {size_in_bytes//1024**2} MB")


Size of my_list: 14 MB


In [85]:
import time
from sklearn.neighbors import KDTree
import numpy as np

points = result

In [ ]:
# kernel = True

In [86]:
# Create a KDTree for each element
element_trees = {}

# Start the timer to measure the query execution time
start_time = time.time()


for element in element_list:
    element_points = [(point['lat'], point['long']) for point in points if point['element'] == element]
    element_trees[element] = KDTree(np.array(element_points))

end_time = time.time()
# Print the time taken to perform the query
print(f"Trees creation time : {end_time - start_time:.4f} seconds")

# Query function
def get_nearby_points(query_point, radius_degrees, element):
    query_array = np.array([query_point])

    # Query the relevant tree for the specified element
    tree = element_trees.get(element)
    if tree:
        indices = tree.query_radius(query_array, r=radius_degrees)
        return [points[i] for i in indices[0]]
    else:
        return []



Trees creation time : 1.9758 seconds


In [87]:
# Get the size of the variable
size_in_bytes = sys.getsizeof(element_trees['Fe']) * 7 

print(f"Size of elements trees: {size_in_bytes//1024} KB")

Size of elements trees: 7 KB


In [104]:
# Query for a specific element
query_point = (66.988234, 22.479195)
radius = 1.0  # 2 degrees radius
element = 'Si'

start_time = time.time()

nearby_points = get_nearby_points(query_point, radius, element)

end_time = time.time()
# Print the time taken to perform the query
print(f"Query time : {end_time - start_time:.4f} seconds")

print(f"{len(nearby_points)} points found")



Query time : 0.0030 seconds
16 points found


In [102]:
N1 = 20
l1 = [[i['lat'], i['long']] for i in nearby_points[:N1]]

l1

[[63.384976, 20.88965],
 [63.568389, 21.611099],
 [63.440604, 20.868272],
 [63.623415, 20.941517],
 [63.715584, 20.629065],
 [63.862185, 19.988494],
 [63.981685, 22.22674],
 [63.957563, 21.489285],
 [63.989492, 20.865799],
 [63.808552, 22.329401],
 [64.198639, 19.929209],
 [64.217924, 21.590061],
 [64.260436, 20.445086],
 [64.406685, 21.841401],
 [64.445796, 20.796922],
 [64.577912, 19.619538],
 [64.639676, 19.551581],
 [64.764734, 19.207426],
 [64.813723, 21.18727],
 [64.874453, 22.78601]]